# Crawler基礎－select、multi-page

## 目錄
- [select](#select)
- [multi-page](#multiple-page)

## 範例網站
- 公視新聞 https://news.pts.org.tw/list/0
- 練習網址 http://smalldragon-r.pancakeapps.com/some_news.html

### select

使用 css selector 來選取元素
- [W3school - CSS_selector](https://www.w3schools.com/cssref/css_selectors.asp)

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# 先把後面會用到的全域變數放在這裡
url = "http://pala.tw/class-id-example/"
headers = {'user-agent':'Safari'}

In [3]:
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [4]:
# 使用 css selector 搜尋
found_list = soup.select('#Pala')
             #soup.find(id='Pala')
found_list

[<p class="無糖 微冰 鐵觀音" id="Pala">Pala的無糖微冰鐵觀音</p>]

In [5]:
# select 不論結果是一個或是多個
# 都會用 list 回傳
found_list = soup.select('.去冰')
found_list

[<p class="半糖 去冰 珍珠 奶茶" id="老闆">老闆的半糖去冰珍珠奶茶</p>,
 <p class="無糖 去冰 綠茶" id="資深前輩">資深前輩的無糖去冰綠茶</p>,
 <p class="半糖 去冰 文山青茶" id="心儀同事">心儀同事的半糖去冰文山青茶</p>]

In [6]:
# 把需要的字串擷取出來
str_list = [each.string for each in found_list]
str_list

['老闆的半糖去冰珍珠奶茶', '資深前輩的無糖去冰綠茶', '心儀同事的半糖去冰文山青茶']

In [7]:
# 取出特別的屬性
get_list = [each.get('id') for each in found_list]
get_list

['老闆', '資深前輩', '心儀同事']

解析HTML的方法差不多就到這裡
常用的三種解析方法：
1. 套件內建的function
2. css selector
3. Xpath
有需要可以再自己上網學習

## multiple-page

如果多練習幾次爬蟲，多使用requests和bs套件  
你會發現他有一些固定的規律  
如果我們想要一次性多抓取一些頁面  
就很適合設計成function  
讓程式碼變的簡潔

In [8]:
# 把 url 傳進來
# 用 requests 抓取後 回傳原始碼
def get_page(url):
    head = {'user-agent':'Safari'}
    res = requests.get(url,headers=head)
    return(res.text)

In [9]:
# 先用 temp 抓取到所有新聞的網頁原始碼
temp = get_page('https://news.pts.org.tw/list/0')

In [10]:
# 利用 bs 選出 title 做成 list
soup = BeautifulSoup(temp,'html.parser')
title_list = soup.find_all(class_='list-news-title') 
title_list = [each.find('a') for each in title_list]
title_list

[<a href="https://news.pts.org.tw/article/378812">不滿勞基法再修法 勞團赴立院抗議</a>,
 <a href="https://news.pts.org.tw/article/378813">勞基法修法續審 綠提限時發言爆衝突</a>,
 <a href="https://news.pts.org.tw/article/378814">修法縮時輪班間隔 護理人員首當其衝</a>,
 <a href="https://news.pts.org.tw/article/378815">入夜後氣溫驟降 北部明低溫下探13度</a>,
 <a href="https://news.pts.org.tw/article/378816">疑受洋流影響 幻象戰機黑盒子訊號消失</a>,
 <a href="https://news.pts.org.tw/article/378817">"王祿仔場"推銷保健品害健康 衛局取締</a>,
 <a href="https://news.pts.org.tw/article/378818">世界大學專業領域排名 台僅"教育"勝中國</a>,
 <a href="https://news.pts.org.tw/article/378819">訂單銳減石斑價格崩盤 轉養又生產過剩</a>,
 <a href="https://news.pts.org.tw/article/378820">整頓"蚊子港" 漁業署擬協助轉型或廢港</a>,
 <a href="https://news.pts.org.tw/article/378821">新南向重大進展 台菲投保協定可望重簽</a>,
 <a href="https://news.pts.org.tw/article/378822">美韓空中聯合軍事演習 規模史上最大</a>,
 <a href="https://news.pts.org.tw/article/378823">日又發現北韓漂流木造船 船上3人死</a>,
 <a href="https://news.pts.org.tw/article/378824">法商登革熱疫苗 未感染者施打恐發病</a>,
 <a href="https://news.pts.org.tw/article

In [11]:
# 選到了 我們用前五個就好
title_list = title_list[0:5]
title_list

[<a href="https://news.pts.org.tw/article/378812">不滿勞基法再修法 勞團赴立院抗議</a>,
 <a href="https://news.pts.org.tw/article/378813">勞基法修法續審 綠提限時發言爆衝突</a>,
 <a href="https://news.pts.org.tw/article/378814">修法縮時輪班間隔 護理人員首當其衝</a>,
 <a href="https://news.pts.org.tw/article/378815">入夜後氣溫驟降 北部明低溫下探13度</a>,
 <a href="https://news.pts.org.tw/article/378816">疑受洋流影響 幻象戰機黑盒子訊號消失</a>]

In [12]:
# 先創個 out 資料夾
from os import makedirs
makedirs('out',exist_ok=True)

In [13]:
for title in title_list:
    res = get_page(title.get('href'))
    
    file_name = 'out/' + title.string + '.html'
    html = open(file_name,'w')
    html.write(res)
    html.close()

In [14]:
# 利用get_page函式 存每個頁面
for title in title_list:
    res = get_page(title.get('href'))
    
    file_name = 'out/' + title.string + '.html'
    html = open(file_name,'w')
    html.write(res)
    html.close()

In [15]:
# 再來寫個負責parser的func吧!
def get_content(html):
    soup = BeautifulSoup(html,'html.parser')
    
    content = soup.find(class_='article-content')
    
    return(content)

In [16]:
for title in title_list:
    file_name = 'out/' + title.string
    
    html = open(file_name + '.html','r')
    content = get_content(html.read())
    html.close()
    
    txt = open(file_name + '.txt','w')
    txt.write(content.get_text())
    txt.close()

In [17]:
for title in title_list:
    file_name = 'out/' + title.string
    
    html = open(file_name + '.html','r')
    content = get_content(html.read())
    html.close()
    
    txt = open(file_name + '.txt','w')
    txt.write(content.get_text())
    txt.close()

In [18]:
# 更快速的方法 不存成檔案 直接放到parser裡面
for title in title_list:
    html = get_page(title.get('href'))
    content = get_content(html)
    
    print(content.get_text()[0:40])
    print()

一例一休勞基法修法草案，今天再度闖關，上百名勞工團體代表，一早就在立法院外高喊抗

一例一休修法今天在立法院續審，一早朝野立委就因為發言時間被限縮為4+2分鐘，而上

勞基法修正草案在立法院審查，場內外都爆發衝突，這次修法打算鬆綁七休一、輪班間隔有



受東北季風影響，今天入夜後氣溫將逐漸下降，明天北部低溫下探13度，中南部1


空軍飛官何子雨駕駛幻象2000戰機失聯，已經快一個月，由於海象不佳，搜救進度



In [19]:
# 世界快
content_list = [get_content(get_page(title.get('href'))).get_text()
               for title in title_list]
content_list

['一例一休勞基法修法草案，今天再度闖關，上百名勞工團體代表，一早就在立法院外高喊抗議口號。中午，勞團三度試圖闖入立院，與警方爆發多次衝突，場面相當混亂。勞團表示，為了擋下修法，已經做好長期抗戰準備，揚言法案審到幾點，就在外面守到幾點，不會離開。\n\r\n拉扯、推擠，喊了一上午的口號，接近中午，勞團代表再也壓不住對例休修法的不滿情緒，一股腦兒的想突破警方封鎖，從群賢樓衝進立法院。\n\r\n警方不敢大意，除了早已用拒馬、柵欄封住立院周遭，也出動了三個分局，共550警力與霹靂小組戒備，一路將勞團與立法院隔開。\n\r\n一陣衝突過後，勞團號召群眾，繞行立法院表達訴求，近下午1點，又爆發第二波與第三波衝撞立院行動，過程中，不少人因此跌倒在地，現場一片混亂。\n\r\n希望在立院裡面的立委，聽到他們的訴求，因為這回修法，放寬七休一、提高加班上限、並縮短輪班間隔時間到8小時，三大方向，只會加重勞工過勞。而且就算政府宣稱有勞資協商、事業主管機關等單位的層層把關，根據過往案例，都只是虛設。\n\r\n美光工會理事長馮澤源表示：「團體協商的前兩個小時，我就被資遣，公司只要把你一個理事長，或那個瓦解掉，讓其他的工會會員，工會幹部的整個信心崩潰，公司就達到這些目的了。」\n\r\n宜蘭縣產業總工會祕書長陳姳臻表示：「勞資協商的部分是擺在優於勞基法，或更高的勞動條件上面，可以使用的一個規則，而不是勞資協商把所有的基礎，跟原則（勞基法）都架空。」\n\r\n經過中午3波衝突後，勞團暫時休息，並由各工會代表依序論述，而為了擋下修法，他們說，已做好長期抗戰準備，法案審到幾點，就在外面守到幾點，不會離開。\n\r\n記者 綜合報導。',
 '一例一休修法今天在立法院續審，一早朝野立委就因為發言時間被限縮為4+2分鐘，而上演全武行。下午推擠衝突還是持續，國民黨和時代力量立委想佔住發言台，也被民進黨立委以人數優勢通通排除。混亂中，召委林靜儀宣布將勞基法24條送交院會討論，強行表決通過。接下來會議要持續到半夜，民進黨力拼闖關成功。\n\r\n立法院喇叭被摔成碎片，混亂中女立委手被抓傷需要醫護人員，勞基法例休修法四號續審，一早，朝野朝野立委就爆發多次肢體衝突，議事不斷停止。\n\r\n民進黨立委邱志偉提出權宜問題，限制發言時間為4+2分鐘，防止冗長發言影響議事，民進黨以多數表決通過，引發在野黨抗議。